**Authors:**  <br>

<div class="alert alert-block alert-success">
    <h3>SCATTER DENSITY DIAGRAM</h3></div>



<hr>This notebook can be used to generate a scatter density diagram (or scatter plot) by selecting a pair of dataset which are distrubuted along with the Climate Station (CS) architecture. The important constraint here is that the two datasets should be comparable on a pixel to pixel basis, i.e. they should have the same spatial resolution, geographical extension and should refer to the same date (for consistency reason).

**The desired dataset can be load from the Climate Station libraries/tools in order to query the CS databese for the Fitness for Purpose (F4P) capabilities. The following workflow show how to get the MODIS-FAPAR together with SPOTV-FAPAR both 10 days 1km data from CS database:**

#### Run the below line to launch the GUI..

In [ ]:
exec("""\nimport os\nimport matplotlib.pyplot as plt\nimport matplotlib.image as mpimg\nfrom apps.c3sf4p.c3s_f4p import Fitness4Purpose\nimport shutil\nfrom osgeo import gdal, osr\nimport rasterio as rio\nfrom rasterio.warp import reproject, Resampling\nimport numpy as np\nfrom lib.python.image_proc.read_write_raster import RasterDatasetCS\n\n\nclass RasterTransformCS:\n    def __init__(self, inputfile, save_path=None, output_name=None, mapset=None, aggregation_method='nearest',\n                 aggregation_resolution=None, coordinates=None):\n        \"\"\"\n        :param inputfile:\n        :param save_path:\n        :param output_name:\n        :param mapset:\n        :param aggregation_method:\n        :param aggregation_resolution:\n        :param coordinates:\n        \"\"\"\n        self.inpname = inputfile\n        self.gobj = gdal.Open(self.inpname)\n        self.outname = output_name\n        self.mapset = mapset\n        self.spath = save_path\n        self.zc = coordinates\n        self.mapset = mapset\n        self.method = aggregation_method\n        self.agg_res = aggregation_resolution\n        if self.agg_res is None:\n            self.agg_res = 0.5\n        if self.zc is None:\n            self.get_coordinates()\n        self.rd = RasterDatasetCS(self.inpname)\n        self.fv = self.rd.fill_value\n        self.of = self.rd.add_offset\n        self.sf = self.rd.scale_factor\n\n    def _resample_gdal(self):\n        \n        method = self.method\n        if method == 'nearest':\n            method = 'near'\n            \n\n        #             W                       S                       E                       N\n        coord_str = str(self.zc[2]) + ' ' + str(self.zc[0]) + ' ' + str(self.zc[3]) + ' ' + str(self.zc[1])\n        gdalwarp = shutil.which("gdalwarp")\n\n        warp_cmd = gdalwarp + ' -overwrite -q -multi -t_srs "+proj=longlat +datum=WGS84" -tr ' + \\\n                   str(self.agg_res) + ' ' + str(self.agg_res) + ' -te ' + coord_str + ' -r ' + method + \\\n            ' -srcnodata ' + str(self.fv) + ' ' + self.inpname + ' ' + self.outname\n\n        os.system(warp_cmd)\n\n    def _get_output_name(self):\n        basename, ext = os.path.splitext(os.path.basename(self.inpname))\n        path0 = os.path.dirname(self.inpname)\n\n        if self.mapset is not None:\n            mapset0 = basename.split('_')[3].split('-')[1] + '-' + basename.split('_')[3].split('-')[2]\n            basename = basename.replace(mapset0, self.mapset)\n        if self.spath is not None:\n            self.outname = self.spath + os.path.sep + basename + ext\n        else:\n            self.outname = path0 + os.path.sep + basename + ext\n\n    def get_coordinates(self):\n\n        old_cs = osr.SpatialReference()\n        old_cs.ImportFromWkt(self.gobj.GetProjectionRef())\n        # create the new coordinate system\n        wgs84_wkt = \"\"\"\n                    GEOGCS["WGS 84",\n                    DATUM["WGS_1984",\n                    SPHEROID["WGS 84",6378137,298.257223563,\n                    AUTHORITY["EPSG","7030"]],\n                    AUTHORITY["EPSG","6326"]],\n                    PRIMEM["Greenwich",0,\n                    AUTHORITY["EPSG","8901"]],\n                    UNIT["degree",0.01745329251994328,\n                    AUTHORITY["EPSG","9122"]],\n                    AUTHORITY["EPSG","4326"]]\"\"\"\n        new_cs = osr.SpatialReference()\n        new_cs.ImportFromWkt(wgs84_wkt)\n        # create a transform object to convert between coordinate systems\n        transform = osr.CoordinateTransformation(old_cs, new_cs)\n\n        # get the point to transform, pixel (0,0) in this case\n        width = self.gobj.RasterXSize\n        height = self.gobj.RasterYSize\n        gt = self.gobj.GetGeoTransform()\n        x = gt[0] + gt[1] * width\n        y = gt[3] + gt[5] * height\n\n        center_x = (gt[0] + x) / 2\n        center_y = (gt[3] + y) / 2\n\n        lon_0, lat_0, _ = transform.TransformPoint(center_x, center_y)\n\n        w, n, _ = transform.TransformPoint(gt[0], gt[3])\n        e, s, _ = transform.TransformPoint(x, y)\n\n        self.zc = [s, n, w, e]\n        return [s, n, w, e]\n\n    def aggregate_to_raster(self):\n        if self.outname is None:\n            self._get_output_name()\n        self._resample_gdal()\n\n        dst_lat = np.arange(self.zc[0], self.zc[1], self.agg_res) + self.agg_res / 2\n        dst_lon = np.arange(self.zc[2], self.zc[3], self.agg_res) + self.agg_res / 2\n\n        dst_shape = (dst_lat.size, dst_lon.size)\n\n        dst_transform = rio.transform.from_bounds(\n            west=dst_lon.min() - self.agg_res / 2,\n            south=dst_lat.min() - self.agg_res / 2,\n            east=dst_lon.max() + self.agg_res / 2,\n            north=dst_lat.max() + self.agg_res / 2,\n            width=dst_shape[1],\n            height=dst_shape[0]\n        )\n\n        src = rio.open(self.inpname)\n        data, _ = reproject(source=src.read(),\n                            destination=np.zeros(dst_shape),\n                            src_transform=src.transform,\n                            dst_transform=dst_transform,\n                            src_crs=src.crs,\n                            dst_crs=src.crs,\n                            dst_nodata=self.fv,\n                            src_nodata=self.fv,\n                            resampling=Resampling.average)\n\n        # get physiscal value and return\n        data = data.astype('float')\n        data[data == self.fv] = np.nan\n        data += self.of\n        data *= self.sf\n\n        return data\n\nfrom ipywidgets import widgets, Layout, TwoByTwoLayout, Box, Checkbox\nfrom IPython.display import display, HTML\nimport datetime\nimport dateutil.relativedelta\nfrom apps.productmanagement.datasets import Dataset\nimport xarray\nfrom matplotlib.colors import ListedColormap,LinearSegmentedColormap\ncolors = ["lightgoldenrodyellow","palegoldenrod","#c7c230","darkkhaki","forestgreen","#34631a","#184a0e","darkred","#e60707","red"]\ncmap1 = LinearSegmentedColormap.from_list("mycmap", colors)\n\ndmax = datetime.datetime.today()\n#dmin = dmax + dateutil.relativedelta.relativedelta(months=-1)\n# dataset1 =Dataset(product_code='modis-fapar', sub_product_code='fapar', version='1.0', mapset='MODIS-Africa-1-1km')\n# dataset2 =Dataset(product_code='vgt-fapar', sub_product_code='fapar', version='V2.0', mapset='SPOTV-Africa-1km')\n\ndatemax= widgets.DatePicker(value=dmax,description='Date to compare:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\n# datemin = widgets.DatePicker(value=dmin,description='Min date:',\n#                              style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\n\nproduct_code_widget = widgets.Text(value='modis-fapar',description='Product code:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\nsub_product_code_widget = widgets.Text(value='fapar',description='Sub Product code:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\nversion_widget = widgets.Text(value='1.0',description='Version:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\nmapset_widget = widgets.Text(value='MODIS-Africa-1-1km',description='Mapset:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\n\nproduct_code_widget1 = widgets.Text(value='vgt-fapar',description='Product code:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\nsub_product_code_widget1 = widgets.Text(value='fapar',description='Sub Product code:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\nversion_widget1 = widgets.Text(value='olci-v1.0',description='Version:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\nmapset_widget1 = widgets.Text(value='SPOTV-Africa-300m',description='Mapset:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='300px', height='32px'))\n\n#param zc:                  geographic extensions, default = [-90S, 90N, -180W, 180E]\nmin_lat_widget = widgets.Text(value='-36',description='Min lat:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\nmax_lat_widget = widgets.Text(value='38',description='Max lat:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\nmax_lon_widget = widgets.Text(value='52',description='Max lon:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\nmin_lon_widget = widgets.Text(value='-19',description='Min lon:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\n\nres_widget = widgets.Text(value='0.05',description='Resolution in degree:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\n\nlayout_2x2_1 = TwoByTwoLayout(top_left=product_code_widget,\n               top_right=sub_product_code_widget,\n               bottom_left=product_code_widget1,\n               bottom_right=sub_product_code_widget1)\n\nlayout_2x2_2 = TwoByTwoLayout(top_left=version_widget,\n               top_right=mapset_widget,\n               bottom_left=version_widget1,\n               bottom_right=mapset_widget1)\n\n\nlayout_2x2_3 = TwoByTwoLayout(top_left=min_lat_widget,\n               top_right=max_lat_widget,\n               bottom_left=min_lon_widget,\n               bottom_right=max_lon_widget)\n\nform_item_layout = Layout(\n    display='flex',\n    flex_flow='column',\n    justify_content='space-between'\n)\nform_items = [\n    Box([layout_2x2_1] , layout=form_item_layout),\n    Box([layout_2x2_2] , layout=form_item_layout)\n    #Box([layout_2x2_3], layout=form_item_layout)\n]\n\nform_items1 = [\n    Box([layout_2x2_3], layout=form_item_layout)\n]\n\nform = Box(form_items, layout=Layout(\n    display='flex',\n    description='Input parameters',\n    flex_flow='row',\n    border='solid 2px',\n    align_items='stretch',\n    width='100%'))\n\nform1 = Box(form_items1, layout=Layout(\n    display='flex',\n    description='Input parameters',\n    flex_flow='row',\n    border='solid 2px',\n    align_items='stretch',\n    width='50%'))\n\nbutton = widgets.Button(\n    description='Scatter Plot',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    layout=widgets.Layout(width='180px', height='32px'),\n    tooltip='Scatter Plot',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\nbutton1 = widgets.Button(\n    description='Plot Upscaled Images',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    layout=widgets.Layout(width='300px', height='32px'),\n    tooltip='Plot Upscaled Images',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n#assign_metadata_box = Checkbox(True, description='Assign metadata')\noutput = widgets.Output()\ndisplay(datemax,form,form1,res_widget, button, output)\n\nout_name1 = None\nout_name2 = None\n\ndef upscale():\n    \n    status = True\n    return status\n\ndef on_button_clicked(b):\n    #status = upscale()\n    # if not status:\n    #     print('Error')\n    #     return\n    status = False\n    product= product_code_widget.value\n    sprod = sub_product_code_widget.value\n    version = version_widget.value\n    mapset = mapset_widget.value\n    \n    product1= product_code_widget1.value\n    sprod1 = sub_product_code_widget1.value\n    version1 = version_widget1.value\n    mapset1 = mapset_widget1.value\n    # Datasets parameters\n    #dmin = datemin.value #datemin.value.strftime('%Y%m%d')\n    # dmax = datemax.value # datemax.value.strftime('%Y%m%d')\n    # from_date = datetime.date(dmin.year, dmin.month, dmin.day)\n    # to_date = datetime.date(dmax.year, dmax.month, dmax.day)\n    kwargs = {\n            'product_code': product,\n            'version': version,\n            'sub_product_code': sprod,\n            'mapset': mapset\n        }\n    dataset1 = Dataset(**kwargs)\n    dataset1.filter(datemax.value.strftime('%Y%m%d'), datemax.value.strftime('%Y%m%d'))\n    \n    kwargs2 = {\n        'product_code': product1,\n        'version': version1,\n        'sub_product_code': sprod1,\n        'mapset': mapset1\n    }\n    dataset2 = Dataset(**kwargs2)\n    dataset2.filter(datemax.value.strftime('%Y%m%d'), datemax.value.strftime('%Y%m%d'))\n    #sst.filter(str(dmin),str(dmax))\n    min_lat = min_lat_widget.value\n    max_lat=max_lat_widget.value\n    min_lon=min_lon_widget.value\n    max_lon=max_lon_widget.value\n    zc = [float(min_lat), float(max_lat), float(min_lon), float(max_lon)]\n    \n    if not (len(dataset1.get_filenames_range()) > 0 and len(dataset2.get_filenames_range()) > 0):\n        print('Date not found') \n        return\n    fn1=dataset1.get_filenames_range()[0]\n    fn2=dataset2.get_filenames_range()[0]\n    \n    res = float(res_widget.value)\n    custom_mapset = 'CommonAfrica-' + str(res) + 'D'\n    \n    seed = os.path.basename(fn1).split('_')\n    out_name1 = seed[0] + '_' + seed[1] + '_' + seed[2] + '_' + custom_mapset + '_' + seed[4]\n\n    seed = os.path.basename(fn2).split('_')\n    out_name2 = seed[0] + '_' + seed[1] + '_' + seed[2] + '_' + custom_mapset + '_' + seed[4]\n\n    rt1 = RasterTransformCS(fn1, output_name=out_name1, aggregation_method='average', aggregation_resolution=res, coordinates=zc)\n    rt2 = RasterTransformCS(fn2, output_name=out_name2, aggregation_method='average', aggregation_resolution=res, coordinates=zc)\n\n    rt1.aggregate_to_raster()\n    rt2.aggregate_to_raster()\n\n    #Append metadata to outputfile\n\n    rd1=RasterDatasetCS(out_name1)\n    data12 = rd1.get_data()\n\n\n    rd2=RasterDatasetCS(out_name2)\n    data21 = rd2.get_data()\n\n    data_orig1 = RasterDatasetCS(fn1).get_data()\n    data_orig2 = RasterDatasetCS(fn2).get_data()\n\n    #The two data are spatially-aggregated at 0.05 within associated metadata in the same size and same geographical extension.\n    print(product+'-'+version+' :',data12.shape)\n    print(product1+'-'+version1+' :',data21.shape)\n    \n    file_list = [[out_name1], [out_name2]]\n\n    bands = [None, None]\n    RC=None\n    ROI='Africa'\n\n    f4p = Fitness4Purpose(file_list, bands, region_name=ROI,region_coordinates=RC)\n\n    [plot, plotfilename] = f4p.scatter_plot(plotimage=True)\n    \n    # data_mask_1=xarray.open_dataset(out_name1)\n    # data_mask_2=xarray.open_dataset(out_name2)\n\n    with output:\n        output.clear_output()\n        img = mpimg.imread(plotfilename)\n        imgplot = plt.imshow(img)\n        plt.show()\n\n\nbutton.on_click(on_button_clicked)\n""")